## Project Context
This notebook documents my individual contribution for the MGMT 467 Term Project, including:
- Prompt logs used during development
- A DIVE analysis for one substantive question
- An interactive Plotly figure based on project data
- A link to the Looker Studio dashboard section I influenced


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import plotly.express as px
import plotly.graph_objects as go


## Dashboard Section Link (What I Influenced)

**Looker Studio Dashboard (View Link):**  
https://lookerstudio.google.com/u/0/reporting/7583f6ff-725b-4507-b47e-4dd24c4c485d/page/ncciF

**Sections Developed:**  
- KPI 1: Real-Time Temperature + Hourly Change (temp_delta_1h)
- City filter behavior (single vs multi-city aggregation explanation)
- Hourly change time-series visualization
- Final Presentation development


## Prompt Logs

Below are the key prompts I used during development and troubleshooting.  
(Each entry includes: Prompt → Tool/Context → Output Summary → Action Taken)

### Prompt Log 1 — KPI feasibility in Looker
**Prompt:**  
"Can Looker Studio compute hour-over-hour temperature change using LAG or window functions directly?"

**Tool/Context:** ChatGPT + Looker Studio calculated fields

**Output Summary:**  
Looker Studio does not support SQL window functions (e.g., LAG) directly in calculated fields for a BigQuery source in the way we needed.

**Action Taken:**  
Moved the hour-over-hour delta calculation into BigQuery (precomputed table/view), then connected that result to Looker.

---

### Prompt Log 2 — BigQuery approach for temp delta
**Prompt:**  
"What is the cleanest BigQuery SQL pattern to compute hourly average temperature and a 1-hour delta per city?"

**Tool/Context:** ChatGPT + BigQuery SQL

**Output Summary:**  
Use hourly bucketing (TIMESTAMP_TRUNC) + AVG() by city/hour; then compute delta using window LAG() over hour.

**Action Taken:**  
Created/used a table like `live_weather_with_delta` with fields:
- city
- ts_hour
- temp_hour_avg
- temp_delta_1h

---

### Prompt Log 3 — Dashboard aggregation behavior
**Prompt:**  
"If the city filter allows multiple selections, why does the line chart look like an average?"

**Tool/Context:** ChatGPT + Looker Studio chart config

**Output Summary:**  
A time series metric with aggregation AVG will average across all selected city series unless you break down by city (or restrict filter to single-select).

**Action Taken:**  
Kept multi-select for the table, but clarified chart behavior; optionally used breakdown = city or limited to single-select for the KPI chart depending on design choice.


## DIVE Entry (Substantive Question)

### D — Define the Question
**Question:**  
How can we compute and visualize hour-over-hour temperature change per city in near real time, given that Looker Studio does not support SQL window functions like LAG directly in calculated fields?

**Why it matters (business relevance):**  
Hour-over-hour temperature change helps users quickly detect short-term shifts (warming/cooling trends) that can impact decisions (travel planning, event prep, frost/heat risk awareness). This KPI is central to the project goal of real-time situational awareness.

---

### I — Investigate (Data + Method)
**Data used:**  
- Streaming `live_weather` table (city, ts, temperature, etc.)
- Derived/precomputed table: `live_weather_with_delta` (hour-bucketed averages + deltas)

**Approach:**  
1. Aggregate observations into hourly buckets per city (to stabilize noisy minute-level readings).  
2. Compute `temp_hour_avg` per `city, ts_hour`.  
3. Compute `temp_delta_1h = temp_hour_avg - LAG(temp_hour_avg)` within each city ordered by hour.  
4. Visualize in Looker Studio using a time-series chart with `ts_hour` on X and `temp_delta_1h` as the metric.

---

### V — Validate (Proof it works)
**Validation checks performed:**
- Confirmed `ts_hour` increments by 1 hour.
- Confirmed delta sign interpretation:
  - Positive delta → warming
  - Negative delta → cooling
  - Near zero → stable
- Spot-checked a few cities manually by comparing consecutive hourly averages.
- Verified Looker time-series updates as new streaming data arrives (refresh behavior).

---

### E — Explain (Conclusion + Next Step)
**Conclusion:**  
Because Looker Studio cannot compute windowed deltas on the fly, the correct design is to precompute hour-over-hour changes in BigQuery and visualize them directly. This keeps the dashboard accurate, fast, and aligned with real-time streaming ingestion.

**Next improvement:**  
Add a categorical status field (Rising/Falling/Stable) using a threshold (e.g., |delta| < 0.2 → Stable) to make the KPI easier to interpret at a glance.


In [ ]:
# Update filename to whatever you exported from BigQuery
df = pd.read_csv("live_weather_with_delta_export.csv")

df.head()


In [ ]:
# Convert ts_hour to datetime if needed
df["ts_hour"] = pd.to_datetime(df["ts_hour"])

# Basic cleanup
df = df.sort_values(["city", "ts_hour"])
df.tail()


In [ ]:
fig = px.line(
    df,
    x="ts_hour",
    y="temp_delta_1h",
    color="city",
    title="Hourly Temperature Change (Δ 1h) by City",
    labels={"ts_hour": "Hour", "temp_delta_1h": "Temp Delta (1h)"}
)

fig.update_layout(
    hovermode="x unified",
    legend_title_text="City"
)

fig.show()


In [ ]:
# Define threshold for "stable"
threshold = 0.2  # adjust if needed

df["temp_trend"] = np.where(
    df["temp_delta_1h"] > threshold, "Rising",
    np.where(df["temp_delta_1h"] < -threshold, "Falling", "Stable")
)

df[["city", "ts_hour", "temp_hour_avg", "temp_delta_1h", "temp_trend"]].head(10)


In [ ]:
fig2 = px.scatter(
    df,
    x="ts_hour",
    y="temp_delta_1h",
    color="temp_trend",
    facet_col="city",
    facet_col_wrap=3,
    title="Hourly Temperature Change with Trend Labels (Rising/Falling/Stable)",
    labels={"ts_hour": "Hour", "temp_delta_1h": "Temp Delta (1h)", "temp_trend": "Trend"}
)

fig2.update_layout(height=800)
fig2.show()


## Summary of My Contribution

- Implemented KPI logic for hour-over-hour change by shifting computation from Looker Studio into BigQuery (window-based delta).
- Connected the derived dataset to Looker Studio and ensured city filter behavior was understood and consistent across visuals.
- Produced an interactive Plotly figure to validate the delta logic outside of Looker and support the final demo narrative.

This notebook provides evidence of the reasoning, technical implementation path, and validation steps behind KPI 1.
